In [1]:
# import word from indonesian-words.txt
import csv
with open('indonesian-words.txt', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    word = [item for sublist in your_list for item in sublist]
    

In [2]:
# read from json
json_path = 'intents_newV2_Combined.json'
import json
with open(json_path) as json_data:
    intents = json.load(json_data)


In [9]:
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import re

# import nltk stopword indonesia from stopword_list.txt
stop_words = []
with open('stopword_list.txt') as f:
        stop_words = f.read().splitlines()

def removeStopwords(text):
        text = text.split(' ')
        text = [x for x in text if x not in stop_words]
        return ' '.join(text)

def removeNonWords(text):
        text = text.split(' ')
        text = [x for x in text if x in word]
        return ' '.join(text)

def stemSentence(sentence):
        return stemmer.stem(sentence)

def cleanTag(text):
        text = text.replace('<p>','').replace('</p>','').replace('<ol>','').replace('</ol>','').replace('<li>','').replace('</li>','').replace('<ul>','').replace('</ul>','').replace('<br>','').replace('br','').replace('<strong>','').replace('</strong>','').replace('<em>','').replace('</em>','').replace('<a href="','').replace('">','').replace('</a>','').replace('<img src="','').replace('" alt="','').replace('" />','').replace('< />','').replace('\xa0','')
        return text

def cleanTagIntent(text):
        text = re.sub(r'[^\w\s]', '',text)
        text = text.lower().replace('keuangan','').replace('kemahasiswaan','').replace('akademik','').strip()
        text = text.replace('_',' ')
        return text


def cleanText(text):
        text = cleanTag(text)
        text = stemSentence(text)
        text = removeStopwords(text)
        text = removeNonWords(text)
        return text

In [10]:
intentsNew = dict()
intentsNew['intents'] = []

for intent in intents['intents']:
    tag = cleanTagIntent(intent['tag'])
    patterns = []
    for pattern in intent['patterns']:
        pattern = cleanText(pattern)
        patterns.append(pattern)
    responses = intent['responses'][0]

    newIntent = {
        'tag': tag,
        'patterns': patterns,
        'responses': [responses]
    }
    intentsNew['intents'].append(newIntent)

In [11]:
# export to newJSON
with open('intents_newV2SiPalingClean_v2.json', 'w') as outfile:
    json.dump(intentsNew, outfile)